In [4]:
! pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 13.5 MB/s eta 0:00:00


In [5]:
import re, random, numpy as np
from collections import Counter
from sklearn.datasets import fetch_20newsgroups

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, ConfusionMatrix

In [6]:
# Map a unique index to each word
words = ["This", "book", "was", "fantastic", "I", "really", "love", "science", "fiction", "but", "the", "protagonist", "was", "rude", "sometimes"]
word_to_idx = {word: i for i, word in enumerate(words)}

# Convert word_to_idx to a tensor
inputs = torch.LongTensor([word_to_idx[w] for w in words])

# Initialize embedding layer with ten dimensions
embedding = nn.Embedding(num_embeddings=len(words), embedding_dim=10)

# Pass the tensor to the embedding layer
output = embedding(inputs)
print(output)

tensor([[-6.0225e-02, -7.2780e-01, -2.9647e-01,  4.8080e-01, -1.6944e+00,
          2.8657e+00,  5.2318e-01,  1.2359e-01, -1.0793e+00, -6.6700e-02],
        [-5.9041e-01, -1.3458e+00,  8.0741e-01,  4.4897e-01,  1.5597e-01,
          4.5849e-01, -3.9183e-02, -1.7998e+00,  9.0804e-01, -6.3406e-02],
        [ 1.0874e+00,  6.6480e-01,  6.0049e-02,  2.3289e-01,  4.2793e-02,
          4.8082e-01, -1.0698e+00, -8.2554e-01, -2.7300e-01,  1.1013e+00],
        [-9.8179e-01, -3.9346e-01,  9.5218e-03,  1.9166e+00, -5.2530e-01,
          9.7612e-01,  1.1070e+00, -3.8642e-01,  2.4127e+00, -1.9479e-01],
        [-1.1190e+00, -7.7903e-01, -5.5399e-01, -1.5482e+00, -3.2684e-01,
          9.7186e-01, -1.0295e-01,  6.3112e-01,  7.8727e-01, -4.3701e-01],
        [-3.0742e-01, -6.1216e-01,  1.5429e+00, -1.5945e+00,  1.0175e+00,
          6.5022e-01, -1.6212e-01, -6.5605e-01,  1.4241e-01,  1.5387e+00],
        [-9.8331e-01,  1.4893e+00,  1.4111e-01,  4.9128e-01, -9.8597e-02,
         -2.2160e+00, -1.1209e+0

TEXT CLASSIFICATION USING CNN

In [7]:
import torch.nn.functional as F

class TextClassificationCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(TextClassificationCNN, self).__init__()
        # Initialize the embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim, embed_dim, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(embed_dim, 2)
    def forward(self, text):
        embedded = self.embedding(text).permute(0, 2, 1)
        # Pass the embedded text through the convolutional layer and apply a ReLU
        conved = F.relu(self.conv(embedded))
        conved = conved.mean(dim=2)
        return self.fc(conved)

In [8]:
vocab = ["i", "love", "this", "book", "do", "not", "like"]
word_to_idx = {word: i for i, word in enumerate(vocab)}
vocab_size = len(word_to_idx)
embed_dim = 10
book_samples = [    ("The story was captivating and kept me hooked until the end.".split(),1),    ("I found the characters shallow and the plot predictable.".split(),0)]
model = TextClassificationCNN(vocab_size, embed_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [9]:
# Define the loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(10):
    # Iterate over book_samples instead of data
    for sentence, label in book_samples:
        # Clear the gradients
        model.zero_grad()
        # Use word_to_idx instead of word_to_ix
        sentence = torch.LongTensor([word_to_idx.get(w, 0) for w in sentence]).unsqueeze(0)
        label = torch.LongTensor([int(label)])
        outputs = model(sentence)
        loss = criterion(outputs, label)
        loss.backward()
        # Update the parameters
        optimizer.step()
print('Training complete!')

Training complete!


This will be a great addition to the PyBooks' recommendation engine, assisting the team in understanding users' sentiments towards different books.

In [10]:
book_reviews = [
    "I love this book".split(),
    "I do not like this book".split()
]
for review in book_reviews:
    # Convert the review words into tensor form, converting to lowercase and handling unknown words
    input_tensor = torch.tensor([word_to_idx.get(w.lower(), 0) for w in review], dtype =torch.long).unsqueeze(0)
    # Get the model's output
    outputs = model(input_tensor)
    # Find the index of the most likely sentiment category
    _, predicted_label = torch.max(outputs.data, 1)
    # Convert the predicted label into a sentiment string
    sentiment = "Positive" if predicted_label.item() == 1 else "Negative"
    print(f"Book Review: {' '.join(review)}")
    print(f"Sentiment: {sentiment}\n")

Book Review: I love this book
Sentiment: Positive

Book Review: I do not like this book
Sentiment: Positive



TEXT CLASSIFICATION USING RNN

In [2]:
import re
from collections import Counter
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import fetch_20newsgroups

# Load the dataset
categories = ['rec.autos', 'sci.med', 'comp.graphics']
X_train_seq = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)
X_test_seq = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state =42)

def preprocess_text(documents):
    preprocessed_docs = []
    for doc in documents:
        # Convert to lowercase
        doc = doc.lower()
        # Remove punctuation and special characters
        doc = re.sub(r'[^a-zA-Z\s]', '', doc)
        # Tokenize
        tokens = doc.split()
        preprocessed_docs.append(tokens)
    return preprocessed_docs

# Apply the preprocessing function to the training and testing data
X_train_tokens = preprocess_text(X_train_seq.data)
X_test_tokens = preprocess_text(X_test_seq.data)

# Create vocabulary and word_to_index mapping from training data
all_train_tokens = [token for doc in X_train_tokens for token in doc]
word_counts = Counter(all_train_tokens)
vocab = [word for word, count in word_counts.most_common()]
word_to_index = {word: i for i, word in enumerate(vocab)}

# Convert tokens to numerical sequences
X_train_numerical = [[word_to_index.get(word, 0) for word in doc] for doc in X_train_tokens]
X_test_numerical = [[word_to_index.get(word, 0) for word in doc] for doc in X_test_tokens]

# Recalculate max_len based on both training and testing sets
max_len = max(max(len(doc) for doc in X_train_numerical), max(len(doc) for doc in X_test_numerical))

X_train_padded = np.zeros((len(X_train_numerical), max_len), dtype=int)
for i, doc in enumerate(X_train_numerical):
    X_train_padded[i, :len(doc)] = doc

X_test_padded = np.zeros((len(X_test_numerical), max_len), dtype=int)
for i, doc in enumerate(X_test_numerical):
    X_test_padded[i, :len(doc)] = doc

# Convert to PyTorch tensors
X_train_tensor = torch.LongTensor(X_train_padded)
X_test_tensor = torch.LongTensor(X_test_padded)

# Convert labels to PyTorch tensors
y_train_tensor = torch.LongTensor(X_train_seq.target)
y_test_tensor = torch.LongTensor(X_test_seq.target)

# Create PyTorch TensorDataset for training and testing data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create PyTorch DataLoader for training and testing datasets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

input_size = len(word_to_index) # Use the actual vocabulary size
embed_dim = 32 # Define embedding dimension
hidden_size = 32
num_layers = 2
num_classes = 3

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim) # Add embedding layer
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(embed_dim, hidden_size, num_layers, batch_first=True) # Change input size to embed_dim
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x) # Pass through embedding layer
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) # Initialize h0 with correct batch size
        out, _ = self.rnn(embedded, h0) # Pass embedded input to RNN
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Initialize the model with vocabulary size and embedding dimension
rnn_model = RNNModel(input_size, embed_dim, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=0.01)

In [7]:
# Set the model to training mode
rnn_model.train()

# Train the model for a few epochs
num_epochs = 5

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = rnn_model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print average loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')

print('Training complete!')

Epoch [1/5], Loss: 1.1190
Epoch [2/5], Loss: 1.1066
Epoch [3/5], Loss: 1.1030
Epoch [4/5], Loss: 1.1019
Epoch [5/5], Loss: 1.1099
Training complete!


TEXT CLASSIFICATION USING LSTM

In [4]:
# Initialize the LSTM and the output layer with parameters
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # Add embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size) # Using hidden_size as embed_dim
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True) # Change input size to hidden_size (embed_dim)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Pass through embedding layer
        embedded = self.embedding(x)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        # Pass embedded input to LSTM
        out, _ = self.lstm(embedded, (h0, c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Initialize model with required parameters
# Use the vocabulary size for input_size
lstm_model = LSTMModel(len(word_to_index), hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.01)

In [9]:
# Set the model to training mode
lstm_model.train()

# Train the model by iterating through the DataLoader
num_epochs = 5

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = lstm_model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print average loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')

print('Training complete!')

Epoch [1/5], Loss: 1.1034
Epoch [2/5], Loss: 1.0999
Epoch [3/5], Loss: 1.1001
Epoch [4/5], Loss: 1.0996
Epoch [5/5], Loss: 1.0992
Training complete!


TEXT CLASSIFICATION USING GRU

In [7]:
# Complete the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, hidden_size) # Add embedding layer
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers,  batch_first=True) # Change input size to hidden_size (embed_dim)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        embedded = self.embedding(x) # Pass through embedding layer
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.gru(embedded, h0) # Pass embedded input to GRU
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Initialize the model
gru_model = GRUModel(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(gru_model.parameters(), lr=0.01)

In [12]:


# Train the model and backpropagate the loss after initialization
# Set the model to training mode
gru_model.train()

# Train the model for a few epochs
num_epochs = 5 # Using 5 epochs for consistency with previous models

for epoch in range(num_epochs):
    total_loss = 0
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = gru_model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print average loss for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')

print('Training complete!')

Epoch [1/5], Loss: 1.1117
Epoch [2/5], Loss: 1.1030
Epoch [3/5], Loss: 1.1019
Epoch [4/5], Loss: 1.1010
Epoch [5/5], Loss: 1.1005
Training complete!


EVALUATION METRICS

In [3]:
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score

# Put model in eval mode
rnn_model.eval()

# Generate predictions
with torch.no_grad():
    outputs = rnn_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

# Create metric instances
accuracy = Accuracy(task="multiclass", num_classes=3)
precision = Precision(task="multiclass", num_classes=3)
recall = Recall(task="multiclass", num_classes=3)
f1 = F1Score(task="multiclass", num_classes=3)

# Ensure everything is on same device
predicted = predicted.cpu()
y_test_tensor = y_test_tensor.cpu()

# Compute metrics (correct argument order)
accuracy_score = accuracy(y_test_tensor, predicted)
precision_score = precision(y_test_tensor, predicted)
recall_score = recall(y_test_tensor, predicted)
f1_score = f1(y_test_tensor, predicted)

print(f"RNN Model - Accuracy: {accuracy_score:.4f}, Precision: {precision_score:.4f}, Recall: {recall_score:.4f}, F1 Score: {f1_score:.4f}")


RNN Model - Accuracy: 0.3362, Precision: 0.3362, Recall: 0.3362, F1 Score: 0.3362


In [8]:
# Create an instance of the metrics
accuracy = Accuracy(task='multiclass', num_classes=3)
precision = Precision(task='multiclass', num_classes=3)
recall = Recall(task='multiclass', num_classes=3)
f1 = F1Score(task='multiclass', num_classes=3)

# Put models in eval mode
lstm_model.eval()
gru_model.eval()

# Generate predictions for LSTM model
with torch.no_grad():
    outputs_lstm = lstm_model(X_test_tensor)
    _, y_pred_lstm = torch.max(outputs_lstm, 1)

# Generate predictions for GRU model
with torch.no_grad():
    outputs_gru = gru_model(X_test_tensor)
    _, y_pred_gru = torch.max(outputs_gru, 1)

# Ensure everything is on same device
y_pred_lstm = y_pred_lstm.cpu()
y_pred_gru = y_pred_gru.cpu()
y_test_tensor = y_test_tensor.cpu() # Use y_test_tensor

# Calculate metrics for the LSTM model
accuracy_1 = accuracy(y_pred_lstm, y_test_tensor)
precision_1 = precision(y_pred_lstm, y_test_tensor)
recall_1 = recall(y_pred_lstm, y_test_tensor)
f1_1 = f1(y_pred_lstm, y_test_tensor)
print("LSTM Model - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(accuracy_1, precision_1, recall_1, f1_1))

# Calculate metrics for the GRU model
accuracy_2 = accuracy(y_pred_gru, y_test_tensor)
precision_2 = precision(y_pred_gru, y_test_tensor)
recall_2 = recall(y_pred_gru, y_test_tensor)
f1_2 = f1(y_pred_gru, y_test_tensor)
print("GRU Model - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1 Score: {:.4f}".format(accuracy_2, precision_2, recall_2, f1_2))

LSTM Model - Accuracy: 0.3294, Precision: 0.3294, Recall: 0.3294, F1 Score: 0.3294
GRU Model - Accuracy: 0.3353, Precision: 0.3353, Recall: 0.3353, F1 Score: 0.3353
